In [2]:
import time
import os 
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd

start_total = time.time()

In [3]:
print("Current directory before change:", os.getcwd())

try:
    
    os.chdir("../")
    print("Current directory after change:", os.getcwd())

except FileNotFoundError:
    print("""
        FileNotFoundError - The specified directory does not exist or you are already in the root.
        If the code already worked once, do not run it again.
    """)

Current directory before change: /home/jacobo/Proyecto_ETL/Notebooks
Current directory after change: /home/jacobo/Proyecto_ETL


In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
import time

start_total = time.time()

publicaciones = pd.read_csv('data/clean/Airbnb_Open_Data_cleaned.csv')   
tiendas = pd.read_csv('data/clean/api_transform.csv')                   

start = time.time()
publicaciones_coords = np.radians(publicaciones[['lat', 'long']].values)
tiendas_coords = np.radians(tiendas[['latitude', 'longitude']].values)
print(f"[Tiempo conversión a radianes] {time.time() - start:.2f} segundos")

start = time.time()
tree = BallTree(tiendas_coords, metric='haversine')
print(f"[Tiempo construcción BallTree] {time.time() - start:.2f} segundos")

start = time.time()
distances, indices = tree.query(publicaciones_coords, k=5)
distances_km = distances * 6371 
print(f"[Tiempo consulta de vecinos] {time.time() - start:.2f} segundos")

start = time.time()
resultados = []
for i, (row_idx, vecino_idxs, dists) in enumerate(zip(publicaciones.index, indices, distances_km)):
    for tienda_idx, dist in zip(vecino_idxs, dists):
        resultados.append({
            'id_publicacion': publicaciones.loc[row_idx, 'id'],
            'id_tienda': tiendas.iloc[tienda_idx]['fsq_id'],
            'distancia_km': round(dist, 4),
            'distancia_km': round(dist, 4),
            'category_group': tiendas.iloc[tienda_idx]['category_group']
        })

resultado_df = pd.DataFrame(resultados)
print(f"[Tiempo ensamblado resultados] {time.time() - start:.2f} segundos")
start = time.time()
resultado_df.to_csv('data/clean/airbnb_top5_nearby.csv', index=False)
print(f"[Tiempo guardado CSV] {time.time() - start:.2f} segundos")
print(f"[Tiempo total de ejecución] {time.time() - start_total:.2f} segundos")


[Tiempo conversión a radianes] 0.01 segundos
[Tiempo construcción BallTree] 0.00 segundos
[Tiempo consulta de vecinos] 1.75 segundos
[Tiempo ensamblado resultados] 43.91 segundos
[Tiempo guardado CSV] 0.63 segundos
[Tiempo total de ejecución] 46.76 segundos


In [13]:
import pandas as pd
import os
import logging
import numpy as np
from rapidfuzz import process, fuzz

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("data_merge.log"), 
        logging.StreamHandler()
    ]
)

logging.info("Inicio del merge de datos.")

try:
    SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    SCRIPT_DIR = os.getcwd()

BASE_DIR = os.path.dirname(os.path.dirname(SCRIPT_DIR))
CLEAN_DATA_DIR = os.path.join(BASE_DIR, 'jacobo', 'Proyecto_ETL', 'data', 'clean')
CSV_AIRBNB_FILENAME = 'Airbnb_Open_Data_cleaned.csv'
CSV_API_FILENAME = 'airbnb_top5_nearby.csv'
CSV_AIRBNB_FILE_PATH = os.path.join(CLEAN_DATA_DIR, CSV_AIRBNB_FILENAME)
CSV_API_FILE_PATH = os.path.join(CLEAN_DATA_DIR, CSV_API_FILENAME)

df_data_airbnb = pd.DataFrame()
df_data_api = pd.DataFrame()

logging.info(f"Ruta del archivo CSV construida con os: {CSV_AIRBNB_FILE_PATH}")
logging.info(f"Ruta del archivo CSV construida con os: {CSV_API_FILE_PATH}")
df_data_merge = pd.DataFrame()

try:
    logging.info(f"Intentando leer los archivos CSV.")
    if not os.path.exists(CSV_AIRBNB_FILE_PATH):
        logging.error(f"Error: Archivo principal no encontrado en '{CSV_AIRBNB_FILE_PATH}'")
        raise FileNotFoundError(f"Archivo principal no encontrado: {CSV_AIRBNB_FILE_PATH}")
    if not os.path.exists(CSV_API_FILE_PATH):
        logging.error(f"Error: Archivo API no encontrado en '{CSV_API_FILE_PATH}'")
        raise FileNotFoundError(f"Archivo API no encontrado: {CSV_API_FILE_PATH}")
    
    df_data_airbnb = pd.read_csv(CSV_AIRBNB_FILE_PATH, low_memory=False)
    df_data_api = pd.read_csv(CSV_API_FILE_PATH, low_memory=False)
    
    logging.info(f"Archivo '{CSV_AIRBNB_FILENAME}' cargado. Shape: {df_data_airbnb.shape}")
    logging.info(f"Archivo '{CSV_API_FILENAME}' cargado. Shape: {df_data_api.shape}")
except FileNotFoundError:
    raise
except Exception as e:
    logging.error(f"Ocurrió un error al cargar los CSV '{CSV_AIRBNB_FILE_PATH}' y '{CSV_API_FILE_PATH}': {e}")
    raise

if not df_data_airbnb.empty or df_data_api.empty:
    logging.info("Verificando filas duplicadas en df_data_airbnb y df_data_api.")
    num_duplicados_airbnb = df_data_airbnb.duplicated().sum()
    num_duplicados_api = df_data_api.duplicated().sum()
    logging.info(f"Número de filas duplicadas encontradas en df_data_airbnb: {num_duplicados_airbnb}")
    logging.info(f"Número de filas duplicadas encontradas en df_data_api: {num_duplicados_api}")
else:
    if df_data_airbnb.empty:
        logging.critical("El DataFrame principal df_data_airbnb está vacío después de la carga. Terminando.")
    if df_data_api.empty:
        logging.critical("El DataFrame API df_data_api está vacío después de la carga. Terminando.")

mapping_api_category = ['restaurants', 'parks_&_outdoor', 'retail_&_shopping', 'entertainment_&_leisure', 'landmarks', 'bars_&_clubs']
logging.info(f"Categorías objetivo para el conteo: {mapping_api_category}")

if not df_data_api.empty and 'id_publicacion' in df_data_api.columns and 'category_group' in df_data_api.columns:
    logging.info("Preparando los datos de API para el merge: contando categorías por id_publicacion.")

    try:
        poi_counts = df_data_api.pivot_table(
            index='id_publicacion',
            columns='category_group',
            values='id_tienda',
            aggfunc='count',
            fill_value=0
        )
        logging.info(f"Tabla pivote de conteo de POIs creada. Shape: {poi_counts.shape}")
    except Exception as e:
        logging.error(f"Error al crear la tabla pivote: {e}", exc_info=True)
        poi_counts = pd.DataFrame()
    
    if not poi_counts.empty:
        for cat in mapping_api_category:
            if cat not in poi_counts.columns:
                poi_counts[cat] = 0
                logging.info(f"Columna '{cat}' añadida a poi_counts con valor 0 (no presente originalmente).")
                
        poi_counts = poi_counts.reindex(columns=mapping_api_category, fill_value=0)
        logging.info(f"Columnas de poi_counts reordenadas y aseguradas. Nuevas columnas: {poi_counts.columns.tolist()}")
        
        new_column_names = {}
        for cat in poi_counts.columns:
            clean_name = cat.replace('_&_', '_and_').replace(' ', '_')
            new_column_names[cat] = f"count_nearby_{clean_name}"
        
        poi_counts.rename(columns=new_column_names, inplace=True)
        logging.info(f"Columnas de conteo de POIs renombradas: {poi_counts.columns.tolist()}")
        
        if 'id' in df_data_airbnb.columns:
            logging.info(f"Realizando left merge entre df_data_airbnb (on 'id') y poi_counts (on index).")
            df_merged = pd.merge(
                df_data_airbnb,
                poi_counts,
                left_on='id',
                right_index=True,
                how='left'
            )
            logging.info(f"Merge completado. Shape de df_merged: {df_merged.shape}")
            
            count_cols_to_fill = poi_counts.columns.tolist()
            for col in count_cols_to_fill:
                if col in df_merged.columns:
                    df_merged[col] = df_merged[col].fillna(0).astype(int)
            logging.info(f"Valores NaN en las nuevas columnas de conteo rellenados con 0 y convertidos a int.")

            logging.info("Primeras filas del DataFrame fusionado (df_merged):")
            logging.info(f"\n{df_merged.head().to_markdown(index=False)}")
            logging.info("Información del DataFrame fusionado:")
            df_merged.info()
            
        else:
            logging.error("La columna 'id' no se encuentra en df_data_airbnb. No se puede realizar el merge.")
            df_merged = df_data_airbnb.copy()

    else:
        logging.warning("La tabla pivote 'poi_counts' está vacía. No se añadirán columnas de conteo.")
        df_merged = df_data_airbnb.copy()

else:
    if df_data_api.empty:
        logging.warning("El DataFrame df_data_api está vacío. No se puede realizar el procesamiento de conteo de categorías.")
    else:
        logging.warning("Las columnas 'id_publicacion' o 'category_group' no existen en df_data_api. No se puede procesar.")
    df_merged = df_data_airbnb.copy()


logging.info("Fin del script de merge de datos.")

2025-05-18 20:14:59,763 - INFO - Inicio del merge de datos.
2025-05-18 20:14:59,765 - INFO - Ruta del archivo CSV construida con os: /home/jacobo/Proyecto_ETL/data/clean/Airbnb_Open_Data_cleaned.csv
2025-05-18 20:14:59,767 - INFO - Ruta del archivo CSV construida con os: /home/jacobo/Proyecto_ETL/data/clean/airbnb_top5_nearby.csv
2025-05-18 20:14:59,769 - INFO - Intentando leer los archivos CSV.


2025-05-18 20:15:00,272 - INFO - Archivo 'Airbnb_Open_Data_cleaned.csv' cargado. Shape: (102222, 22)
2025-05-18 20:15:00,273 - INFO - Archivo 'airbnb_top5_nearby.csv' cargado. Shape: (511110, 4)
2025-05-18 20:15:00,274 - INFO - Verificando filas duplicadas en df_data_airbnb y df_data_api.
2025-05-18 20:15:00,402 - INFO - Número de filas duplicadas encontradas en df_data_airbnb: 0
2025-05-18 20:15:00,403 - INFO - Número de filas duplicadas encontradas en df_data_api: 0
2025-05-18 20:15:00,404 - INFO - Categorías objetivo para el conteo: ['restaurants', 'parks_&_outdoor', 'retail_&_shopping', 'entertainment_&_leisure', 'landmarks', 'bars_&_clubs']
2025-05-18 20:15:00,405 - INFO - Preparando los datos de API para el merge: contando categorías por id_publicacion.
2025-05-18 20:15:00,521 - INFO - Tabla pivote de conteo de POIs creada. Shape: (102222, 8)
2025-05-18 20:15:00,523 - INFO - Columnas de poi_counts reordenadas y aseguradas. Nuevas columnas: ['restaurants', 'parks_&_outdoor', 'reta

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102222 entries, 0 to 102221
Data columns (total 28 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   id                                      102222 non-null  int64  
 1   name                                    102222 non-null  object 
 2   host_id                                 102222 non-null  int64  
 3   host_name                               102222 non-null  object 
 4   neighbourhood_group                     102222 non-null  object 
 5   neighbourhood                           102222 non-null  object 
 6   lat                                     102222 non-null  float64
 7   long                                    102222 non-null  float64
 8   cancellation_policy                     102222 non-null  object 
 9   room_type                               102222 non-null  object 
 10  construction_year                       1022

In [14]:
df_merged.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,lat,long,cancellation_policy,room_type,...,calculated_host_listings_count,availability_365,host_verification,instant_bookable_flag,count_nearby_restaurants,count_nearby_parks_and_outdoor,count_nearby_retail_and_shopping,count_nearby_entertainment_and_leisure,count_nearby_landmarks,count_nearby_bars_and_clubs
0,1,Clean & quiet apt home by the park,150000,Madaline,Brooklyn,Kensington,40.64749,-73.97237,strict,Private room,...,6,286,False,False,0,2,1,0,1,1
1,2,Skylit Midtown Castle,150001,Jenna,Manhattan,Midtown,40.75362,-73.98377,moderate,Entire home/apt,...,2,228,True,False,0,4,0,1,0,0
2,3,THE VILLAGE OF HARLEM....NEW YORK !,150002,Elise,Manhattan,Harlem,40.80902,-73.94190,flexible,Private room,...,1,352,False,True,0,0,0,0,5,0
3,4,no_name,150003,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,moderate,Entire home/apt,...,1,322,False,True,0,1,2,0,2,0
4,5,Entire Apt: Spacious Studio/Loft by central park,150004,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,moderate,Entire home/apt,...,1,289,True,False,0,0,0,0,4,1


In [ ]:
pip list | grep great-expectations

python -m great_expectations.cli suite new

great_expectations suite edit raw_api_suite

/usr/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=157316) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


great-expectations                       1.4.4
Note: you may need to restart the kernel to use updated packages.
